# Book Preprocessing and Chunking

## 1. Introduction and Preliminary Work

This notebook details the first step in our analysis - dataset creation. We decided to collect our own dataset, using the following steps:
- Selected books from feminist writers that are publicly available on the Gutenberg library website
- Colleted titles, authors, year of release and afferent link in `book_links.csv` (data/)
- Extracted each book's content into a text file using `extract_data.py` (src/)
- .txt files are saved in (data/raw/), along with `metadata.csv` (data/)

These .txt files are not included in our final deliverable, as we included their cleaned version, following after Step 3 in this notebook. All the other mentioned files can be found in the (extra/) folder we sent.

Thus, our task is to carefully process the books, removing unnecessary content, normalizing the texts, tokenizing and splitting them into chunks. The final outputs are 2 datasets, `processed_book_chunks.csv` and `processed_book_chunks_context.csv` (data/processed/) which will then be used for our 3-part analysis.

**Summary of Preprocessing and Chunking:**
- We cleaned the text files by removing the standard Guutenberg heather and footer
- We normalized line endings and whitespaces
- We manually cleaned the text contents that could not be removed in a standard way (data/cleaned/)
- We performed chunking with and without context, tokenizing and splitting sections in batches for efficient use
- The 2 outputs are dataframes that include text chunk (~500 words), title, author, year of release and chunk ID

## 2. Setup and File Paths

In [4]:
# standard libraries
from pathlib import Path
import pandas as pd
import spacy
import re
import shutil
import os
from tqdm import tqdm

# load NLP model (can later be swapped for en_core_web_trf, for now we'll start with this a lighter one)
nlp = spacy.load("en_core_web_sm")

/Users/emmamora/anaconda3/envs/feminist-nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# set base repository path (assumes notebook is in repo/notebooks/)
REPO_DIR = Path(".").resolve().parents[1]

# key data paths
RAW_DATA_DIR = REPO_DIR / "data" / "raw"
PROCESSED_DIR = REPO_DIR / "data" / "processed"
BOOK_LINKS_PATH = REPO_DIR / "data" / "book_links.csv"
METADATA_PATH = REPO_DIR / "data" / "metadata.csv"

# confirm setup
print("Repository Path:", REPO_DIR)
print("Raw Data Directory:", RAW_DATA_DIR)
print("Processed Book Chunk Output:", PROCESSED_DIR)
print("Metadata CSV:", METADATA_PATH)

Repository Path: /Users/emmamora/Documents/GitHub/feminist_nlp
Raw Data Directory: /Users/emmamora/Documents/GitHub/feminist_nlp/data/raw
Processed Book Chunk Output: /Users/emmamora/Documents/GitHub/feminist_nlp/data/processed
Metadata CSV: /Users/emmamora/Documents/GitHub/feminist_nlp/data/metadata.csv


## 3. Clean Raw Text: Gutenberg Add-ons and Normalize

At this stage, we process the raw .txt files to remove boilerplate text added by Project Gutenberg and normalize formatting. The goals are:
- Remove standardized headers and footers using regular expressions.
- Normalize inconsistent newlines, tabs, and spacing for cleaner input downstream.
- Attempt to isolate the actual content of the book, excluding prefaces, introductions, chapter indices, etc.

We implemented a `clean_gutenberg_text()` function to handle the standardized parts of the Gutenberg text. It performs:
- Header removal using *** START OF THIS PROJECT GUTENBERG EBOOK ... ***
- Footer removal using *** END OF THIS PROJECT GUTENBERG EBOOK ... ***
- Normalization of line breaks and collapsing of excess whitespace

However, this was **not sufficient for all files**:
- Many books include content before the actual novel begins (e.g., prefaces, illustrations, chapter listings)
- These vary widely across titles and could not be reliably removed with a universal pattern
- For example, “Pride and Prejudice” includes a full preface that the regex cannot detect

Because of this variability, we decided:
- To manually inspect and clean the start of each file
- This ensures only the actual narrative content is preserved
- Manual cleaning was done on the files saved (data/cleaned/)

After this:
- We proceed with tokenization and chunking
- Further NLP steps such as lemmatization were not applied at this stage to keep text closer to original form

The cleaned files will now serve as input for downstream chunking and analysis tasks.

In [4]:
def clean_gutenberg_text(text: str) -> str:
    # remove Gutenberg header
    text = re.split(r'\*\*\* START OF (THE|THIS) PROJECT GUTENBERG EBOOK .* \*\*\*', text, flags=re.IGNORECASE)[-1]

    # remove Gutenberg footer
    text = re.split(r'\*\*\* END OF (THE|THIS) PROJECT GUTENBERG EBOOK .* \*\*\*', text, flags=re.IGNORECASE)[0]

    # normalize line endings and whitespace
    text = text.replace('\r\n', '\n').replace('\r', '\n')          # normalize newlines
    text = re.sub(r'\n{2,}', '\n\n', text)                         # collapse multiple empty lines to 2
    text = re.sub(r'[ \t]+', ' ', text)                            # remove multiple spaces/tabs
    text = text.strip()                                            # trim leading/trailing whitespace

    return text

In [5]:
# test file
sample_txt_path = RAW_DATA_DIR / "The_Yellow_Wallpaper.txt"

# load and clean it
with open(sample_txt_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

cleaned_text = clean_gutenberg_text(raw_text)

# show result
print("Original length:", len(raw_text))
print("Cleaned length :", len(cleaned_text))
print("\n--- Cleaned Sample ---\n")
print(cleaned_text[:1000])

Original length: 31602
Cleaned length : 31496

--- Cleaned Sample ---

The Yellow Wallpaper

By Charlotte Perkins Gilman

It is very seldom that mere ordinary people like John and myself secure
ancestral halls for the summer.

A colonial mansion, a hereditary estate, I would say a haunted house,
and reach the height of romantic felicity—but that would be asking too
much of fate!

Still I will proudly declare that there is something queer about it.

Else, why should it be let so cheaply? And why have stood so long
untenanted?

John laughs at me, of course, but one expects that in marriage.

John is practical in the extreme. He has no patience with faith, an
intense horror of superstition, and he scoffs openly at any talk of
things not to be felt and seen and put down in figures.

John is a physician, and perhaps—(I would not say it to a living soul,
of course, but this is dead paper and a great relief to my
mind)—perhaps that is one reason I do not get well faster.

You see, he does not

In [6]:
# test file
sample_txt_path = RAW_DATA_DIR / "Pride_and_Prejudice.txt"

# load and clean it
with open(sample_txt_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

cleaned_text = clean_gutenberg_text(raw_text)

# show result
print("Original length:", len(raw_text))
print("Cleaned length :", len(cleaned_text))
print("\n--- Cleaned Sample ---\n")
print(cleaned_text[:1000])

Original length: 728842
Cleaned length : 721387

--- Cleaned Sample ---

[Illustration:

 GEORGE ALLEN
 PUBLISHER

 156 CHARING CROSS ROAD
 LONDON

 RUSKIN HOUSE
 ]

 [Illustration:

 _Reading Jane’s Letters._ _Chap 34._
 ]

 PRIDE.
 and
 PREJUDICE

 by
 Jane Austen,

 with a Preface by
 George Saintsbury
 and
 Illustrations by
 Hugh Thomson

 [Illustration: 1894]

 Ruskin 156. Charing
 House. Cross Road.

 London
 George Allen.

 CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
 TOOKS COURT, CHANCERY LANE, LONDON.

 [Illustration:

 _To J. Comyns Carr
 in acknowledgment of all I
 owe to his friendship and
 advice, these illustrations are
 gratefully inscribed_

 _Hugh Thomson_
 ]

PREFACE.

[Illustration]

_Walt Whitman has somewhere a fine and just distinction between “loving
by allowance” and “loving with personal love.” This distinction applies
to books as well as to men and women; and in the case of the not very
numerous authors who are the objects of the personal affection, it
brings

In [6]:
# create clean directory in data, we will copy-paste the raw data and manually clean it
CLEANED_DATA_DIR = REPO_DIR / "data" / "cleaned"
CLEANED_DATA_DIR.mkdir(parents=True, exist_ok=True)
print(f"Cleaned directory: {CLEANED_DATA_DIR}")

Cleaned directory: /Users/emmamora/Documents/GitHub/feminist_nlp/data/cleaned


In [10]:
# loop through each .txt file and clean + save
for txt_file in RAW_DATA_DIR.glob("*.txt"):
    with open(txt_file, "r", encoding="utf-8") as f:
        raw_text = f.read()

    cleaned_text = clean_gutenberg_text(raw_text)

    cleaned_file_path = CLEANED_DATA_DIR / txt_file.name
    with open(cleaned_file_path, "w", encoding="utf-8") as f:
        f.write(cleaned_text)

    print(f"Cleaned: {txt_file.name}")

Cleaned: Herland.txt
Cleaned: The_Voyage_Out.txt
Cleaned: The_Womans_Bible.txt
Cleaned: A_Vindication_of_the_Rights_of_Woman.txt
Cleaned: Narrative_of_Sojourner_Truth.txt
Cleaned: Woman_in_the_Nineteenth_Century.txt
Cleaned: Strife_and_Peace.txt
Cleaned: Mrs_Dalloway.txt
Cleaned: Wuthering_Heights.txt
Cleaned: An_OldFashioned_Girl.txt
Cleaned: The_Mill_on_the_Floss.txt
Cleaned: The_Yellow_Wallpaper.txt
Cleaned: Eighty_Years_and_More.txt
Cleaned: Emma.txt
Cleaned: Pride_and_Prejudice.txt
Cleaned: The_Awakening.txt
Cleaned: Little_Women.txt
Cleaned: Mary_A_Fiction.txt
Cleaned: Middlemarch.txt
Cleaned: Bayou_Folk.txt
Cleaned: Jane_Eyre.txt


## 4. Chunking: Process Text into Final Dataset

After cleaning, we proceed with tokenizing and chunking each book’s content to prepare for analysis. The goal is to transform long book texts into manageable, context-rich units (~500 words each), while preserving relevant metadata for each chunk.

**Steps Overview:**
- Load the metadata from metadata.csv and align it with cleaned .txt files
- Define functions for sentence tokenization and word-based chunking
- Tokenize using spaCy in batches (to efficiently handle large books like Middlemarch)
- Generate two datasets:
	- `processed_book_chunks_context.csv`: with overlapping 50-word context between chunks
	- `processed_book_chunks.csv`: without overlap (independent chunks)
	- Store for downstream use in analysis notebooks

**Tokenization and Chunking Functions**
- `tokenize_sentences()`:
	- Splits the text into paragraphs
	- Uses spaCy’s pipe() method to batch-process and tokenize paragraphs into sentences
- `chunk_sentences()`:
	- Groups tokenized sentences into ~500 word chunks
	- Allows optional overlap between chunks to preserve continuity
	- Appends metadata (title, author, year, chunk ID) for each unit

**Execution**

We ran the pipeline twice:
1. With 50-word overlap (`processed_book_chunks_context.csv`): Ensures semantic continuity across adjacent chunks, useful for contextual models.
2.	Without overlap (`processed_book_chunks.csv`): Ensures strict independence between chunks, useful for classification or indexing tasks.

Both versions include:
- `chunk`: the raw text of the chunk
- `title`: book title
- `author`: author name
- `year`: publication year
- `chunk_id`: a unique identifier formatted as AuthorInitials_ChunkNumber

This structure forms the foundation of our subsequent exploratory and modeling notebooks.

In [13]:
# set path for final dataset
OUTPUT_PATH_CONTEXT = PROCESSED_DIR / "processed_book_chunks_context.csv"
OUTPUT_PATH_NO_CONTEXT = PROCESSED_DIR / "processed_book_chunks.csv"

# load metadata
metadata = pd.read_csv(METADATA_PATH)
metadata.set_index("filename", inplace=True)

# checks
print("Metadata loaded. Example:")
display(metadata.head())

Metadata loaded. Example:


,id,title,author,year
filename,,,,
Pride_and_Prejudice.txt,Pride_and_Prejudice,Pride and Prejudice,Jane Austen,1813
Emma.txt,Emma,Emma,Jane Austen,1815
Jane_Eyre.txt,Jane_Eyre,Jane Eyre,Charlotte Brontë,1847
Wuthering_Heights.txt,Wuthering_Heights,Wuthering Heights,Emily Brontë,1847
Middlemarch.txt,Middlemarch,Middlemarch,George Eliot,1871


In [15]:
# functions for tokenization and chunking

def tokenize_sentences(text, batch_size=20, show_progress=False):
    # tokenizes text into sentences using spaCy's pipe in batches => prevents memory overload on large books like Middlemarch
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    # show progress bar if requested
    if show_progress:
        paragraphs = tqdm(paragraphs, desc="Paragraphs")
    sentences = []
    for doc in nlp.pipe(paragraphs, batch_size=batch_size):
        sentences.extend([sent.text.strip() for sent in doc.sents if sent.text.strip()])
    return sentences

def chunk_sentences(sentences, max_words=500, overlap=0, debug=False):
    # split list of sentences into word-limited chunks, optionally preserving overlap for context retention
    chunks = []
    current_chunk = []
    current_word_count = 0

    for sent in sentences:
        words_in_sent = len(sent.split())

        # if adding this sentence exceeds max_words, save current chunk
        if current_word_count + words_in_sent > max_words:
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # reset, but include overlap from end of previous chunk
            if overlap > 0:
                overlap_tokens = " ".join(current_chunk).split()[-overlap:]
                current_chunk = [" ".join(overlap_tokens)]
                current_word_count = len(overlap_tokens)
            else:
                current_chunk = []
                current_word_count = 0

        current_chunk.append(sent)
        current_word_count += words_in_sent

    # add the final chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # debug first 3 chunks (optional)
    if debug:
        for i, chunk in enumerate(chunks[:3]):
            print(f"\n--- Chunk {i+1} (words: {len(chunk.split())}) ---\n{chunk[:300]}...\n")

    return chunks

In [9]:
for book_file in CLEANED_DATA_DIR.glob("*.txt"):
    book_name = book_file.name
    print(f"Processing {book_name}...") 


Processing Herland.txt...
Processing The_Voyage_Out.txt...
Processing The_Womans_Bible.txt...
Processing A_Vindication_of_the_Rights_of_Woman.txt...
Processing Narrative_of_Sojourner_Truth.txt...
Processing Woman_in_the_Nineteenth_Century.txt...
Processing Strife_and_Peace.txt...
Processing Mrs_Dalloway.txt...
Processing Wuthering_Heights.txt...
Processing An_OldFashioned_Girl.txt...
Processing The_Mill_on_the_Floss.txt...
Processing The_Yellow_Wallpaper.txt...
Processing Eighty_Years_and_More.txt...
Processing Emma.txt...
Processing Pride_and_Prejudice.txt...
Processing The_Awakening.txt...
Processing Little_Women.txt...
Processing Mary_A_Fiction.txt...
Processing Middlemarch.txt...
Processing Bayou_Folk.txt...
Processing Jane_Eyre.txt...


In [11]:
# process all cleaned books => CONTEXT CHUNKS
all_chunk_data = []

book_files = list(CLEANED_DATA_DIR.glob("*.txt"))

for book_file in tqdm(book_files, desc="Processing books"):
    book_name = book_file.name
    if book_name not in metadata.index:
        print(f"Skipping {book_name} (no metadata)")
        continue

    with open(book_file, "r", encoding="utf-8") as f:
        text = f.read()

    # tokenize
    sentences = tokenize_sentences(text, show_progress=False)

    # chunk
    chunks = chunk_sentences(sentences, overlap=50, debug=True)

    # metadata
    meta = metadata.loc[book_name]
    title, author, year = meta["title"], meta["author"], meta["year"]
    author_code = "".join([w[0].upper() for w in author.split()[:2]])

    # collect chunks
    for i, chunk in enumerate(chunks):
        chunk_id = f"{author_code}_{i+1:03d}"
        all_chunk_data.append({
            "chunk": chunk,
            "title": title,
            "author": author,
            "year": year,
            "chunk_id": chunk_id
        })

print(f"Finished processing: {len(all_chunk_data)} total chunks from {len(book_files)} books.")

Processing books:   5%|▍         | 1/21 [00:06<02:13,  6.68s/it]


--- Chunk 1 (words: 466) ---
CHAPTER 1.
A Not Unnatural Enterprise This is written from memory, unfortunately. If I could have brought with
me the material I so carefully prepared, this would be a very different
story. Whole books full of notes, carefully copied records, firsthand
descriptions, and the pictures--that’s the wors...


--- Chunk 2 (words: 499) ---
few things that don’t. We three had a chance to join a big scientific expedition. They needed a doctor, and that gave Jeff an excuse for dropping his just opening practice; they needed Terry’s experience, his machine, and his money; and as for me, I got in through Terry’s influence. The expedition w...


--- Chunk 3 (words: 499) ---
I had understood, so I showed him a red and blue pencil I carried, and asked again. Yes, he pointed to the river, and then to the southwestward. “River--good water--red and blue.” Terry was close by and interested in the fellow’s pointing. “What does he say, Van?” I told him. Terry blazed up at once

Processing books:  10%|▉         | 2/21 [00:23<04:03, 12.81s/it]


--- Chunk 1 (words: 475) ---
CHAPTER I As the streets that lead from the Strand to the Embankment are very
narrow, it is better not to walk down them arm-in-arm. If you persist,
lawyers’ clerks will have to make flying leaps into the mud; young lady
typists will have to fidget behind you. In the streets of London where
beauty g...


--- Chunk 2 (words: 482) ---
case they should proceed to tease his wife, Mr. Ambrose flourished his stick at them, upon which they decided that he was grotesque merely, and four instead of one cried “Bluebeard!” in chorus. Although Mrs. Ambrose stood quite still, much longer than is natural, the little boys let her be. Some one...


--- Chunk 3 (words: 447) ---
already occupied by two city men. The fixity of her mood was broken by the action of walking. The shooting motor cars, more like spiders in the moon than terrestrial objects, the thundering drays, the jingling hansoms, and little black broughams, made her think of the world she lived in. Somewhere u

Processing books:  14%|█▍        | 3/21 [00:39<04:16, 14.26s/it]

...


--- Chunk 1 (words: 494) ---
PRELUDE. Who that cares much to know the history of man, and how the mysterious
mixture behaves under the varying experiments of Time, has not dwelt,
at least briefly, on the life of Saint Theresa, has not smiled with
some gentleness at the thought of the little girl walking forth one
morning hand-i...


--- Chunk 2 (words: 491) ---
the living stream in fellowship with its own oary-footed kind. Here and there is born a Saint Theresa, foundress of nothing, whose loving heart-beats and sobs after an unattained goodness tremble off and are dispersed among hindrances, instead of centring in some long-recognizable deed. BOOK I. MISS...


--- Chunk 3 (words: 500) ---
by heart; and to her the destinies of mankind, seen by the light of Christianity, made the solicitudes of feminine fashion appear an occupation for Bedlam. She could not reconcile the anxieties of a spiritual life involving eternal consequences, with a keen interest in gimp and artificial protrusion

Processing books:  95%|█████████▌| 20/21 [04:22<00:16, 16.01s/it]


--- Chunk 1 (words: 476) ---
A NO-ACCOUNT CREOLE I. One agreeable afternoon in late autumn two young men stood together on
Canal Street, closing a conversation that had evidently begun within
the club-house which they had just quitted. "There's big money in it, Offdean," said the elder of the two. "I would
n't have you touch it...


--- Chunk 2 (words: 488) ---
business man may be said alternately to exist, and which reduce him, naturally, to a rather ragged condition of soul. Offdean had done, in a temperate way, the usual things which young men do who happen to belong to good society, and are possessed of moderate means and healthy instincts. He had gone...


--- Chunk 3 (words: 500) ---
in a slovenly fashion, but so rich that cotton and corn and weed and "cocoa-grass" grew rampant if they had only the semblance of a chance. The negro quarters were at the far end of this open stretch, and consisted of a long row of old and very crippled cabins. Directly back of these a dense wood gr

Processing books: 100%|██████████| 21/21 [04:45<00:00, 13.60s/it]


--- Chunk 1 (words: 498) ---
CHAPTER I There was no possibility of taking a walk that day. We had been
wandering, indeed, in the leafless shrubbery an hour in the morning;
but since dinner (Mrs. Reed, when there was no company, dined early)
the cold winter wind had brought with it clouds so sombre, and a rain
so penetrating, th...


--- Chunk 2 (words: 487) ---
I was, I could not pass quite as a blank. They were those which treat of the haunts of sea-fowl; of “the solitary rocks and promontories” by them only inhabited; of the coast of Norway, studded with isles from its southern extremity, the Lindeness, or Naze, to the North Cape— “Where the Northern Oce...


--- Chunk 3 (words: 450) ---
soon. The breakfast-room door opened. “Boh! Madam Mope!” cried the voice of John Reed; then he paused: he found the room apparently empty. “Where the dickens is she!” he continued. “Lizzy! Georgy! (calling to his sisters) Joan is not here: tell mama she is run out into the rain—bad animal!” “It is w

In [12]:
# save chunks to csv
df_chunks = pd.DataFrame(all_chunk_data)
df_chunks.to_csv(OUTPUT_PATH_CONTEXT, index=False)

print(f"Final dataset saved to: {OUTPUT_PATH_CONTEXT}")
display(df_chunks.head())

Final dataset saved to: /Users/emmamora/Documents/GitHub/feminist_nlp/data/processed/processed_book_chunks.csv


,chunk,title,author,year,chunk_id
0,CHAPTER 1.\nA Not Unnatural Enterprise This is...,Herland,Charlotte Perkins Gilman,1915,CP_001
1,few things that don’t. We three had a chance t...,Herland,Charlotte Perkins Gilman,1915,CP_002
2,"I had understood, so I showed him a red and bl...",Herland,Charlotte Perkins Gilman,1915,CP_003
3,glass and squatted down to investigate. “Chemi...,Herland,Charlotte Perkins Gilman,1915,CP_004
4,a dozen Montenegroes up and down these great r...,Herland,Charlotte Perkins Gilman,1915,CP_005


In [16]:
# process books => NO CONTEXT CHUNKS
all_chunk_data = []

book_files = list(CLEANED_DATA_DIR.glob("*.txt"))

for book_file in tqdm(book_files, desc="Processing books"):
    book_name = book_file.name
    if book_name not in metadata.index:
        print(f"Skipping {book_name} (no metadata)")
        continue

    with open(book_file, "r", encoding="utf-8") as f:
        text = f.read()

    # tokenize
    sentences = tokenize_sentences(text, show_progress=False)

    # chunk
    chunks = chunk_sentences(sentences, debug=True)

    # metadata
    meta = metadata.loc[book_name]
    title, author, year = meta["title"], meta["author"], meta["year"]
    author_code = "".join([w[0].upper() for w in author.split()[:2]])

    # collect chunks
    for i, chunk in enumerate(chunks):
        chunk_id = f"{author_code}_{i+1:03d}"
        all_chunk_data.append({
            "chunk": chunk,
            "title": title,
            "author": author,
            "year": year,
            "chunk_id": chunk_id
        })

print(f"Finished processing: {len(all_chunk_data)} total chunks from {len(book_files)} books.")

Processing books:   5%|▍         | 1/21 [00:06<02:12,  6.60s/it]


--- Chunk 1 (words: 466) ---
CHAPTER 1.
A Not Unnatural Enterprise This is written from memory, unfortunately. If I could have brought with
me the material I so carefully prepared, this would be a very different
story. Whole books full of notes, carefully copied records, firsthand
descriptions, and the pictures--that’s the wors...


--- Chunk 2 (words: 500) ---
The expedition was up among the thousand tributaries and enormous
hinterland of a great river, up where the maps had to be made, savage
dialects studied, and all manner of strange flora and fauna expected. But this story is not about that expedition. That was only the merest
starter for ours. My int...


--- Chunk 3 (words: 499) ---
It was early yet; we had just breakfasted; and leaving word that we’d
be back before night, we got away quietly, not wishing to be thought
too gullible if we failed, and secretly hoping to have some nice little
discovery all to ourselves. It was a long two hours, nearer three. I fancy the savage cou

Processing books:  10%|▉         | 2/21 [00:23<04:03, 12.82s/it]


--- Chunk 1 (words: 475) ---
CHAPTER I As the streets that lead from the Strand to the Embankment are very
narrow, it is better not to walk down them arm-in-arm. If you persist,
lawyers’ clerks will have to make flying leaps into the mud; young lady
typists will have to fidget behind you. In the streets of London where
beauty g...


--- Chunk 2 (words: 499) ---
Some one is always looking into the river
near Waterloo Bridge; a couple will stand there talking for half an
hour on a fine afternoon; most people, walking for pleasure,
contemplate for three minutes; when, having compared the occasion with
other occasions, or made some sentence, they pass on. Some...


--- Chunk 3 (words: 474) ---
She knew how to read the people who were passing her; there were the
rich who were running to and from each others’ houses at this hour;
there were the bigoted workers driving in a straight line to their
offices; there were the poor who were unhappy and rightly malignant. Already, though there was s

Processing books:  14%|█▍        | 3/21 [00:40<04:21, 14.53s/it]

...


--- Chunk 1 (words: 494) ---
PRELUDE. Who that cares much to know the history of man, and how the mysterious
mixture behaves under the varying experiments of Time, has not dwelt,
at least briefly, on the life of Saint Theresa, has not smiled with
some gentleness at the thought of the little girl walking forth one
morning hand-i...


--- Chunk 2 (words: 441) ---
Since I can do no good because a woman,
Reach constantly at something that is near it. —_ The Maid’s Tragedy:_ BEAUMONT AND FLETCHER. Miss Brooke had that kind of beauty which seems to be thrown into
relief by poor dress. Her hand and wrist were so finely formed that she
could wear sleeves not less ...


--- Chunk 3 (words: 463) ---
Her mind was theoretic, and yearned
by its nature after some lofty conception of the world which might
frankly include the parish of Tipton and her own rule of conduct there;
she was enamoured of intensity and greatness, and rash in embracing
whatever seemed to her to have those aspects; likely to s

Processing books:  95%|█████████▌| 20/21 [04:21<00:15, 15.86s/it]


--- Chunk 1 (words: 476) ---
A NO-ACCOUNT CREOLE I. One agreeable afternoon in late autumn two young men stood together on
Canal Street, closing a conversation that had evidently begun within
the club-house which they had just quitted. "There's big money in it, Offdean," said the elder of the two. "I would
n't have you touch it...


--- Chunk 2 (words: 492) ---
He had gone to college, had traveled a
little at home and abroad, had frequented society and the clubs, and
had worked in his uncle's commission-house; in all of which employments
he had expended much time and a modicum of energy. But he felt all through that he was simply in a preliminary stage
of ...


--- Chunk 3 (words: 494) ---
A dozen rods or more from the Red River bank stood the dwelling-house,
and nowhere upon the plantation had time touched so sadly as here. The
steep, black, moss-covered roof sat like an extinguisher above the
eight large rooms that it covered, and had come to do its office so
poorly that not more th

Processing books: 100%|██████████| 21/21 [04:44<00:00, 13.56s/it]


--- Chunk 1 (words: 498) ---
CHAPTER I There was no possibility of taking a walk that day. We had been
wandering, indeed, in the leafless shrubbery an hour in the morning;
but since dinner (Mrs. Reed, when there was no company, dined early)
the cold winter wind had brought with it clouds so sombre, and a rain
so penetrating, th...


--- Chunk 2 (words: 497) ---
“Where the Northern Ocean, in vast whirls,
Boils round the naked, melancholy isles
Of farthest Thule; and the Atlantic surge
Pours in among the stormy Hebrides.” Nor could I pass unnoticed the suggestion of the bleak shores of
Lapland, Siberia, Spitzbergen, Nova Zembla, Iceland, Greenland, with
“the...


--- Chunk 3 (words: 497) ---
And I came out immediately, for I trembled at the idea of being dragged
forth by the said Jack. “What do you want?” I asked, with awkward diffidence. “Say, ‘What do you want, Master Reed?’” was the answer. “I want you to
come here;” and seating himself in an arm-chair, he intimated by a
gesture that

In [17]:
# save chunks to csv
df_chunks = pd.DataFrame(all_chunk_data)
df_chunks.to_csv(OUTPUT_PATH_NO_CONTEXT, index=False)

print(f"Final dataset saved to: {OUTPUT_PATH_NO_CONTEXT}")
display(df_chunks.head())

Final dataset saved to: /Users/emmamora/Documents/GitHub/feminist_nlp/data/processed/processed_book_chunks.csv


,chunk,title,author,year,chunk_id
0,CHAPTER 1.\nA Not Unnatural Enterprise This is...,Herland,Charlotte Perkins Gilman,1915,CP_001
1,The expedition was up among the thousand tribu...,Herland,Charlotte Perkins Gilman,1915,CP_002
2,It was early yet; we had just breakfasted; and...,Herland,Charlotte Perkins Gilman,1915,CP_003
3,“Woman Country--up\nthere.” Then we were inter...,Herland,Charlotte Perkins Gilman,1915,CP_004
4,"It had a special covering of fitted armor, thi...",Herland,Charlotte Perkins Gilman,1915,CP_005
